In [35]:
import glob, os, json, re, unicodedata
from bs4 import BeautifulSoup


# load JSON text files; parsing into raw text
directory = "../data/unprocessed/"
#text_dir = "../data/processed3/"
orig_dir = "../data/original/"
clean_dir = "../data/clean/"

count = 0
for filename in glob.glob(os.path.join(directory, '*.json')):
        json_data=open(filename)
        data = json.load(json_data)
        print filename
        for data2 in data["speeches"]:
            speaker = data2['speaker']

            if ('News Conference With' not in data2['name']) and ('News Conference with' not in data2['name']) and (' With President ' not in data2['name']):
                count += 1

                #if (speaker == "Barack Obama"):
                #    print speaker, data2['name'], data2['date']

                name = speaker.split(' ')
                
                if ('W. Bush' in speaker):
                    pres_lastname = 'WBush'
                else:
                    pres_lastname = name[len(name) - 1]

                outfile = format('%s_NewsConf%s_%s.txt' % (pres_lastname, count, data2['date']))
                # Remove ' ' and comma
                outfile = re.sub('[\s,\,]','', outfile, flags=re.DOTALL)
                
                    
                orig_file = open(orig_dir + outfile, 'w')
                clean_file = open(clean_dir + outfile, 'w')
                # data2['text'] has a lot of htmtl tags in there. We still need to parse it
                raw = data2['text']
                raw = BeautifulSoup(raw, "html.parser").get_text(" ")
                raw = re.sub(u"\u2014",' - ', raw, flags=re.DOTALL)
                #if (outfile == 'Roosevelt_NewsConf431_April211942.txt'):
                #    print raw #data2['text']

                raw = unicodedata.normalize('NFKD', raw).encode('ascii','ignore')


                orig_file.write(raw)
                orig_file.close()
                
                
                # Remove []
                raw = re.sub(' \[.*?\]',' ', raw, flags=re.DOTALL)
                # Remove ()
                raw = re.sub(' \(.*?\)',' ', raw, flags=re.DOTALL)
                # Removing the questions
                raw = re.sub('[A-Z,\s,\.]Q\..*? The President\.',' ',raw, flags=re.DOTALL)
                raw = re.sub('[A-Z,\s,\.]Q\..*?THE PRESIDENT\.',' ',raw, flags=re.DOTALL)

                raw = re.sub('^Q\..*?THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)
                raw = re.sub('[\s,\.]Q\..*?THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)

                raw = re.sub('[\s]Reporter:.*?THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)

                raw = re.sub('[\s]Joseph H\. Short :.*?THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)
                raw = re.sub('[\s]Mr\. Short:.*?THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)
                raw = re.sub('[\s]Mr\. Davis:.*?THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)


                raw = re.sub('[\s]THE PRESIDENT[:,;,\.]',' ',raw, flags=re.DOTALL)

                raw = re.sub('^THE PRESIDENT[:,;,\.]','',raw, flags=re.DOTALL)
                
#                print "writing data to", outfile
                clean_file.write(raw)
                clean_file.close()

                
        json_data.close()
print "Count =", count

                

../data/unprocessed/Presidential_News_Confs_(1929-2017).json
../data/unprocessed/Presidential_News_Confs_(2009-).json
Count = 1717


### After that, we need to do some more clean up from the unix environment

grep -l "^ Citation:" *txt | awk '{print "sed -i -e \"/^ Citation/d\" " $1 }' > x

./x

Remove "NOTE: ....":  

grep -l " NOTE: " *txt | awk '{print "sed \"s/ NOTE:.*//\" " $1 " > y; mv y " $1}' > x

grep -l " Note: " *txt | awk '{print "sed \"s/ Note:.*//\" " $1 " > y; mv y " $1}' > x

Replace "\." with newline: grep -l " \\\. " *txt | awk '{print "sed  \"s/\\\\\\./\\'\\\n'/g\" " $1 " > x; mv x " $1}' > z

grep -l "^The President. " *txt > x
cat x | awk '{print "sed \"s/^The President.//g\" " $1 " > y; mv y " $1}' > z

Remove "--": grep -l "\-\- \-\-" *txt | awk '{print "sed  \"s/\\-\\- \\-\\-//g\" " $1 " > x; mv x " $1}' > z
